## 테스트. (LangChain을 이용한 AI 에이전트 만들기)

### 프로그램 개요
이메일을 스팸, 환불요구, 단순문의로 분류하고 답장 초안을 작성한다. 일단 이메일은 테스트용으로 문자열로 입력한다.

### 목표
1. Google Gemini API 키를 받아와 설정
2. LangChain 구성요소 (노드 함수작성, Workflow 구성) 만들어 보기

### Step 1. 패키지 설치

In [2]:
!pip install langgraph langchain-google-genai pydantic

   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 14.1 MB/s  0:00:00
   ---------------------------------------- 0.0/713.3 kB ? eta -:--:--
   ---------------------------------------- 713.3/713.3 kB 7.3 MB/s  0:00:00

   - --------------------------------------  1/40 [zstandard]
   -- -------------------------------------  2/40 [xxhash]
   --- ------------------------------------  3/40 [websockets]
   --- ------------------------------------  3/40 [websockets]
   ---- -----------------------------------  4/40 [uuid-utils]
   ----- ----------------------------------  5/40 [urllib3]
  Attempting uninstall: typing-extensions
   ----- ----------------------------------  5/40 [urllib3]
    Found existing installation: typing_extensions 4.12.2
   ----- ----------------------------------  5/40 [urllib3]
    Uninstalling typing_extensions-4.12.2:
   ----- ----------------------------------  5/40 [urllib3]
   ------ -----

### Step 2. Google Gemini API 키 설정

In [ ]:
import os
from typing import TypedDict, Literal
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

# 1. API 키 설정 (Google Gemini API 키 필요)
os.environ["GOOGLE_API_KEY"] = "your-google-api-key"  # 여기에 Google API Key 입력

# 2. 모델 초기화 (Gemini 모델 사용)
llm = ChatGoogleGenerativeAI(model="gemini-flash-lite-latest", temperature=0)

### Step 3. LangChain 노드 정의

In [6]:
# 3. 상태(State) 정의: 노드 간에 주고받을 데이터 구조
class EmailState(TypedDict):
    email_content: str      # 원본 이메일 내용
    category: str           # 분류된 카테고리
    draft: str              # 생성된 답장 초안

# 4. 노드 함수 정의 (에이전트의 행동)

# [Node 1] 이메일 분류
def classify_email(state: EmailState):
    print("--- 1. 이메일 분류 중 ---")
    prompt = ChatPromptTemplate.from_template(
        "다음 이메일을 분석하여 'SPAM', 'REFUND', 'INQUIRY' 중 하나로 분류해줘. 오직 단어 하나만 출력해.\n\n이메일: {email}"
    )
    chain = prompt | llm
    category = chain.invoke({"email": state["email_content"]}).content.strip()
    print(f"분류 결과: {category}")
    return {"category": category}

# [Node 2] 답장 초안 작성
def draft_reply(state: EmailState):
    print("--- 2. 답장 초안 작성 중 ---")
    category = state["category"]
    email = state["email_content"]
    
    if category == "SPAM":
        return {"draft": "IGNORE"}
    
    prompt = ChatPromptTemplate.from_template(
        "당신은 친절한 고객 지원 AI입니다. 다음 {category} 유형의 이메일에 대한 정중한 답장을 작성해주세요.\n\n원문: {email}"
    )
    chain = prompt | llm
    draft = chain.invoke({"category": category, "email": email}).content
    return {"draft": draft}

### Step 4. LangChain Workflow 구성 및 컴파일

In [7]:

# 5. 그래프(Workflow) 구성
workflow = StateGraph(EmailState)

# 노드 추가
workflow.add_node("classifier", classify_email)
workflow.add_node("drafter", draft_reply)

# 엣지(흐름) 연결
workflow.set_entry_point("classifier") # 시작점
workflow.add_edge("classifier", "drafter") # 분류 -> 작성
workflow.add_edge("drafter", END) # 작성 -> 종료

# 컴파일
app = workflow.compile()


### Step 5. 실행 및 테스트

In [8]:
# 6. 실행 및 테스트
test_email = """
안녕하세요, 지난주에 구매한 커피 머신이 작동하지 않습니다. 
전원이 켜지지 않아요. 환불이나 교환을 받고 싶습니다.
주문 번호는 12345입니다.
"""

print(f"입력 이메일: {test_email}")
result = app.invoke({"email_content": test_email})

print("\n================ 최종 결과 ================")
print(f"카테고리: {result['category']}")
print(f"생성된 초안:\n{result['draft']}")

입력 이메일: 
안녕하세요, 지난주에 구매한 커피 머신이 작동하지 않습니다. 
전원이 켜지지 않아요. 환불이나 교환을 받고 싶습니다.
주문 번호는 12345입니다.

--- 1. 이메일 분류 중 ---
분류 결과: REFUND
--- 2. 답장 초안 작성 중 ---

================ 최종 결과 ================
카테고리: REFUND
생성된 초안:
안녕하세요, 고객님. 저희 제품으로 인해 불편을 드려 정말 죄송합니다.

주문하신 커피 머신이 작동하지 않아 많이 당황스러우셨을 것 같습니다. 고객님의 불편을 최소화하기 위해 신속하게 도움을 드리겠습니다.

주문 번호 **12345**로 접수된 건 확인했습니다.

환불 또는 교환 중 원하시는 처리 방법을 알려주시면, 해당 절차를 바로 진행해 드리겠습니다.

**환불 또는 교환 절차 안내:**

1. **제품 회수:** 저희가 지정한 택배사를 통해 불량 제품을 회수할 예정입니다. 회수 절차를 위해 고객님의 성함과 연락 가능한 전화번호를 회신해 주시면 감사하겠습니다.
2. **처리:** 제품이 저희 물류센터에 도착하는 대로 요청하신 **환불** 또는 **교환** 처리를 진행하겠습니다.

번거로우시겠지만, 원하시는 처리 방법(환불 또는 교환)과 함께 연락처를 다시 한번 회신해 주시면 감사하겠습니다.

다시 한번 불편을 드린 점 사과드리며, 최대한 빠르게 문제를 해결해 드릴 수 있도록 최선을 다하겠습니다.

감사합니다.

[귀사 이름] 고객 지원팀 드림
